In [ ]:
# 1. Mount Google Drive (run once)
from google.colab import drive
drive.mount('/content/drive')

# 2. Import required libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tqdm import tqdm
from warnings import filterwarnings
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard

filterwarnings('ignore')

# 3. Define data paths
data_root = '/content/drive/MyDrive/498milestones'
train_dir = os.path.join(data_root, 'Training')
test_dir  = os.path.join(data_root, 'Testing')


In [ ]:
# 4. Visualize color palettes
colors_dark  = ["#1F1F1F","#313131","#636363","#AEAEAE","#DADADA"]
colors_red   = ["#331313","#582626","#9E1717","#D35151","#E9B4B4"]
colors_green = ["#01411C","#4B6F44","#4F7942","#74C365","#D0F0C0"]
sns.palplot(colors_dark)
sns.palplot(colors_green)
sns.palplot(colors_red)

# 5. Load data
labels = ['glioma_tumor', 'no_tumor', 'meningioma_tumor', 'pituitary_tumor']
image_size = 150
X, y = [], []

for label in labels:
    folder = os.path.join(train_dir, label)
    for fname in tqdm(os.listdir(folder), desc=f'Loading TRAIN/{label}'):
        img = cv2.imread(os.path.join(folder, fname))
        img = cv2.resize(img, (image_size, image_size))
        X.append(img)
        y.append(label)

for label in labels:
    folder = os.path.join(test_dir, label)
    for fname in tqdm(os.listdir(folder), desc=f'Loading TEST/{label}'):
        img = cv2.imread(os.path.join(folder, fname))
        img = cv2.resize(img, (image_size, image_size))
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)
print(f'Total samples: {X.shape[0]}, Image dimensions: {X.shape[1:]}')

# 6. Display sample images for each class
fig, axes = plt.subplots(1, 4, figsize=(20, 20))
fig.suptitle('Sample Images From Each Label', fontsize=18)
for idx, label in enumerate(labels):
    sample_index = np.where(y == label)[0][0]
    axes[idx].imshow(cv2.cvtColor(X[sample_index], cv2.COLOR_BGR2RGB))
    axes[idx].set_title(label)
    axes[idx].axis('off')


In [ ]:
# 7. Shuffle and split into training/test sets
X, y = shuffle(X, y, random_state=101)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=101
)

# 8. Encode labels to one-hot
def to_onehot(labels_list):
    indices = [labels.index(lbl) for lbl in labels_list]
    return tf.keras.utils.to_categorical(indices, num_classes=len(labels))

y_train = to_onehot(y_train)
y_test  = to_onehot(y_test)

# 9. Build transfer learning model
base_model = EfficientNetB0(
    weights='imagenet', include_top=False,
    input_shape=(image_size, image_size, 3)
)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
out = tf.keras.layers.Dense(len(labels), activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=out)
model.summary()

# 10. Compile model and set callbacks (with ModelCheckpoint)
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
callbacks = [
    TensorBoard(log_dir='logs'),
    ReduceLROnPlateau(monitor='val_accuracy', factor=0.3, patience=2, verbose=1),
    ModelCheckpoint(
        filepath=os.path.join(data_root, 'model_best.h5'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# 11. Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=12,
    batch_size=32,
    verbose=1,
    callbacks=callbacks
)

# 12. Save the final model after training
model.save(os.path.join(data_root, 'model_final.h5'))



In [ ]:
# 13. Plot training curves
epochs = range(1, len(history.history['accuracy']) + 1)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
ax1.plot(epochs, history.history['accuracy'], label='Train Acc')
ax1.plot(epochs, history.history['val_accuracy'], label='Val Acc')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.legend()
ax2.plot(epochs, history.history['loss'], label='Train Loss')
ax2.plot(epochs, history.history['val_loss'], label='Val Loss')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.legend()
plt.show()

# 14. Evaluate on test set
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred, target_names=labels))

plt.figure(figsize=(8, 6))
sns.heatmap(
    confusion_matrix(y_true, y_pred),
    xticklabels=labels, yticklabels=labels,
    annot=True, fmt='d', cmap='Blues'
)
plt.title('Confusion Matrix')
plt.show()
